In [1]:
import numpy as np 
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.transforms as transforms

In [2]:
data = pd.read_csv('/content/train.csv')
print(data.shape)

(20653, 785)


In [3]:
y = data['label'].values
X = data.drop(['label'],1).values
X = X.reshape(-1,28,28,1)/255.0

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [5]:
X_train.shape

(17555, 28, 28, 1)

In [6]:
class CustomDataset(Dataset):
    def __init__(self, X, y, transforms=False):
        self.X, self.y = X , y
        self.transforms = transforms
    def __len__(self):
        return len(self.X)    
    def __getitem__(self, idx):        
        return self.transforms(self.X[idx]) , self.y[idx]

preprocess = transforms.Compose([
    transforms.ToTensor(),   
])

In [7]:
# create data loaders
bs = 32
train_loader =  DataLoader(CustomDataset(X_train, y_train,  preprocess), batch_size = bs, shuffle=True)
test_loader  =  DataLoader(CustomDataset(X_test, y_test,  preprocess), batch_size = bs)

In [8]:
for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
    print(batch_idx,X_batch.shape,y_batch.shape)

0 torch.Size([32, 1, 28, 28]) torch.Size([32])
1 torch.Size([32, 1, 28, 28]) torch.Size([32])
2 torch.Size([32, 1, 28, 28]) torch.Size([32])
3 torch.Size([32, 1, 28, 28]) torch.Size([32])
4 torch.Size([32, 1, 28, 28]) torch.Size([32])
5 torch.Size([32, 1, 28, 28]) torch.Size([32])
6 torch.Size([32, 1, 28, 28]) torch.Size([32])
7 torch.Size([32, 1, 28, 28]) torch.Size([32])
8 torch.Size([32, 1, 28, 28]) torch.Size([32])
9 torch.Size([32, 1, 28, 28]) torch.Size([32])
10 torch.Size([32, 1, 28, 28]) torch.Size([32])
11 torch.Size([32, 1, 28, 28]) torch.Size([32])
12 torch.Size([32, 1, 28, 28]) torch.Size([32])
13 torch.Size([32, 1, 28, 28]) torch.Size([32])
14 torch.Size([32, 1, 28, 28]) torch.Size([32])
15 torch.Size([32, 1, 28, 28]) torch.Size([32])
16 torch.Size([32, 1, 28, 28]) torch.Size([32])
17 torch.Size([32, 1, 28, 28]) torch.Size([32])
18 torch.Size([32, 1, 28, 28]) torch.Size([32])
19 torch.Size([32, 1, 28, 28]) torch.Size([32])
20 torch.Size([32, 1, 28, 28]) torch.Size([32])
21

<img src="https://github.com/neurospin/pystatsml/blob/master/deep_learning/figures/LeNet_Original_Image.jpg"/>

In [9]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5,stride=1)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #print('x',self.conv1(x).shape)
        x = F.max_pool2d(F.relu(self.conv1(x)),2,2)
        #print(x.shape)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2,2)
        x = x.view(-1, self.num_flat_features(x))           
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [10]:
model = Net()

In [11]:
print(model)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [12]:
for p in model.parameters():
    print(p.shape)

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [13]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.001) #lr=0.001
error = nn.CrossEntropyLoss()
EPOCHS = 2

In [14]:
def fit(model, train_loader):
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            X_batch = X_batch.float()           
            optimizer.zero_grad()
            output = model(X_batch)
            loss = error(output, y_batch)
            loss.backward()
            optimizer.step()
            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == y_batch).sum()
            if batch_idx % 100 == 0:
                print('Epoch : {},\t Accuracy:{:.3f}%'.format(epoch,correct*100/float(bs*(batch_idx+1))))

In [15]:
fit(model, train_loader)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch : 0,	 Accuracy:6.250%
Epoch : 0,	 Accuracy:9.963%
Epoch : 0,	 Accuracy:10.199%
Epoch : 0,	 Accuracy:9.790%
Epoch : 0,	 Accuracy:9.702%
Epoch : 0,	 Accuracy:9.911%
Epoch : 1,	 Accuracy:12.500%
Epoch : 1,	 Accuracy:10.458%
Epoch : 1,	 Accuracy:10.137%
Epoch : 1,	 Accuracy:9.749%
Epoch : 1,	 Accuracy:9.609%
Epoch : 1,	 Accuracy:9.837%
